# Probability

* 베이지안 통계의 기본은 베이즈 이론이고, 베이즈 이론의 기본은 조건부 확률이다. 
* 베이즈 이론을 사용해 조건부확률과 관련된 문제를 다뤄보고 베이즈 이론을 베이지안 통계로 전환하는 내용은 추후에 다루기로 한다.

## Linda the Banker

To introduce conditional probability, I'll use an example from a [famous experiment by Tversky and Kahneman](https://en.wikipedia.org/wiki/Conjunction_fallacy), who posed the following question:
* 아래의 질문에 답한다면? (린다를 설명하는 문장 중 더 가능성 있는 말은?)

> Linda is 31 years old, single, outspoken, and very bright. She majored in philosophy. As a student, she was deeply concerned with issues of discrimination and social justice, and also participated in anti-nuclear demonstrations.  Which is more probable?
> 1. Linda is a bank teller.
> 2. Linda is a bank teller and is active in the feminist movement.

* 대부분의 사람들은 두 번째를 답으로 선택하지만, 두 번째 답이 질문에서 요구하는 것처럼 더 가능성이 있을 수는 없다.


If you were inclined to choose the second option, you are in good company.  The biologist [Stephen J. Gould wrote](https://doi.org/10.1080/09332480.1989.10554932) :

> I am particularly fond of this example because I know that the [second] statement is least probable, yet a little [homunculus](https://en.wikipedia.org/wiki/Homunculus_argument) in my head continues to jump up and down, shouting at me, "but she can't just be a bank teller; read the description."

If the little person in your head is still unhappy, maybe this chapter will help.

## Probability
* 확률(probability) 에 대해 정의는 의외로 [까다롭다](https://en.wikipedia.org/wiki/Probability_interpretations).
* 쉽게 진행하기 위해 확률은 유한한 집합에서의 비율로 생각해보자
* 예를 들어, 1000명의 사람들에 대해 설문조사를 하는데, 그 중 20명이 은행원이었다면, 은행원의 비율은 0.02 혹은 2%다. 이 1000명의 사람 중에 임의의 한 사람을 골랐을 때, 그 사람이 은행원일 확률은 2%다. '임의로'에 내포된 것은 모든 사람이 선택될 가능성이 같다는 점이다.
* [General Social Survey](http://gss.norc.org/) (GSS). 로 예시를 살펴보자

The following cell downloads the data.

In [1]:
# Load the data file

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/gss_bayes.csv')

Downloaded gss_bayes.csv


I'll use Pandas to read the data and store it in a `DataFrame`.

In [2]:
import pandas as pd

gss = pd.read_csv('gss_bayes.csv', index_col=0)
gss.head()

,year,age,sex,polviews,partyid,indus10
caseid,,,,,,
1,1974,21.0,1,4.0,2.0,4970.0
2,1974,41.0,1,5.0,0.0,9160.0
5,1974,58.0,2,6.0,1.0,2670.0
6,1974,30.0,1,5.0,4.0,6870.0
7,1974,48.0,1,5.0,4.0,7860.0


### Data Description
* `caseid`: Respondent id (which is the index of the table).

* `year`: Year when the respondent was surveyed.

* `age`: Respondent's age when surveyed.

* `sex`: Male or female.

* `polviews`: Political views on a range from liberal to conservative.

* `partyid`: Political party affiliation, Democrat, Independent, or Republican.

* `indus10`: [Code](https://www.census.gov/cgi-bin/sssd/naics/naicsrch?chart=2007) for the industry the respondent works in.
    * 산업 코드에 대한 부분은 link 참조

## Fraction of Bankers
* 은행원의 비율은 아래의 코드로 알 수 있다.

In [3]:
banker = (gss['indus10'] == 6870)
banker.head()

caseid
1    False
2    False
5    False
6     True
7    False
Name: indus10, dtype: bool

* row 단위로 은행원 여부에 대해 bool 값을 가지는 Series 가 return 되어서 결과에 대해 집계 함수를 사용하면 원하는 결과를 얻을 수 있다.

In [4]:
banker.sum()

728

* 728 명의 은행원이 있고 이에 대한 비율은 mean 함수로 얻을 수 있다.

In [6]:
banker.mean()

0.014769730168391155

About 1.5% of the respondents work in banking, so if we choose a random person from the dataset, the probability they are a banker is about 1.5%.

## The Probability Function

* 앞에서 얻은 boolean 값을 가지는 Series 를 넣을 때 그 확률을 얻는 함수를 아래와 같이 지정

In [7]:
def prob(A):
    """Computes the probability of a proposition, A."""    
    return A.mean()

So we can compute the fraction of bankers like this:

In [8]:
prob(banker)

0.014769730168391155

* 성별의 변수를 똑같이 적용해보면 아래와 같이 가능하다. 여성 (female) 일 확률은 똑같이 Series 를 생성한다.
```
1    Male
2    Female
```

In [9]:
female = (gss['sex'] == 2)

And use it to compute the fraction of respondents who are women.

In [10]:
prob(female)

0.5378575776019476

* The fraction of women in this dataset is higher than in the adult U.S. population because [the GSS does not include people living in institutions](https://gss.norc.org/faq) like prisons and military housing, and those populations are more likely to be male.

## Political Views and Parties

정치적 성향을 나타내는 `polviews` 와 소속 정당을 나타내는 `partyid` 를 살펴보자.

The values of `polviews` are on a seven-point scale:

```
1	매우 진보적
2	진보적
3	약간 진보적
4	중립
5	조금 보수적
6	보수적
7	매우 보수적
```

* `liberal` 은 '매우 진보적', '진보적', '조금 진보적' 이라고 응답한 경우 (True) 로 정의

In [11]:
liberal = (gss['polviews'] <= 3)

* 응답자가 진보적 정치 성향을 가질 확률은 아래와 같다.

In [13]:
prob(liberal)

0.27374721038750255

* 이 Dataset 을 기준으로 진보적 정치 성향의 응답자가 선택될 확률은 27% 이다.

The values of `partyid` are encoded like this:

```
0	적극적 민주당원
1	보통 민주당원
2	무소속이나 민주당 지지
3	무소속
4	무소속이나 공화당 지지
5	보통 공화당원
6	적극적 공화당원
7	다른 정당원
```

* `democrat` 를 적극적 민주당원 이거나 보통 민주당원인 응답자를 포함한다 정의

In [14]:
democrat = (gss['partyid'] <= 1)

* 민주당원이라 응답한 사람의 비중은 아래와 같다.

In [15]:
prob(democrat)

0.3662609048488537

## Conjunction

* 앞서 확률을 정의하고 이를 계산한 함수를 만들었으니, 논리곱(Conjunction) 에 대해 알아보자
* "Conjunction" 은 `and` operation 의 다른 이름이다.  만약 두 개의 [명제](https://en.wikipedia.org/wiki/Proposition), `A` 와 `B`가 있을때, A와 B가 모두 참이면 A와 B의 논리곱은 참이고, 아닌 경우는 거짓이다. 두 개의 boolean Series 에 & 연산자를 추가하면 쉽게 논리곱을 구할 수 있다.
* 예를 들어 응답자가 은행원이면서 민주당원일 확률은 아래와 같다.

In [16]:
prob(banker)

0.014769730168391155

In [17]:
prob(democrat)

0.3662609048488537

In [18]:
prob(banker & democrat)

0.004686548995739501

* 예상할 수 있듯, `prob(banker & democrat)` 은 `prob(banker)` 보다 작다. 모든 은행원이 민주당우너이 아니기 때문이다.
* 또한 논리곱은 교환 법칙이 성립한다. (that is, `A & B` should be the same as `B & A`, To check, we can also compute `prob(democrat & banker)`)

In [19]:
prob(democrat & banker)

0.004686548995739501

## Conditional Probability

* 조건부 확률은 조건에 따른 확률이지만 한번에 와닿는 정의는 아니다. 아래의 예시를 보자.

    * 한 응답자가 진보적이라고 했을 때, 이 응답자가 민주당원일 확률은?

    * 한 응답자가 은행원일 때, 이 응답자가 여성일 확률은?

    * 한 응답자가 여성일 때, 이 응답자가 진보적일 확률은?

* 첫 번째 질문의 경우 "모든 진보 성향 응답자 중, 민주당원의 비율은 얼마인가?" 로 치환이 가능하다. (즉, 비율의 비율로 생각할 수 있다)
* 이를 아래의 두 단계로 계산이 가능하다.
1. 모든 진보 성향의 응답자를 골라낸다.
2. 이 중 민주당원의 비율을 구한다.
* 대괄호 [] 를 사용해 진보 성향 응답자를 골라낸다.

In [20]:
selected = democrat[liberal]

`selected` 변수는 진보 성향 응답자 중에 `democrat`, 즉 민주당원인 경우을 포함하므로 `prob(selected)` 진보 성향 응답자 중 민주당원의 비율을 계산한다.

In [21]:
prob(selected)

0.5206403320240125

* 진보 성향인 사람 중 절반 조금 넘는 사람들이 민주당원이다.  이 값이 예상보다 낮다면 다음의 내용을 고려해보자:

1. 민주당원 이라는 말을 엄격하게 정의하여 민주당 '지지자' 는 데이터에 반영되지 않음.

2. 이 데이터셋은 1974년의 응답자까지만 포함한다. 수집 기간 중 초기에는 오늘날에 비해 정치관과 정당 참여 정도가 덜 일치했다.

* 질문을 바꿔, 한 응답자가 은행원일 때 이 응답자가 여성일 확률을 구하려고 할 떄, 이 문제는 모든 은행원 중 여성의 비율은 얼마인가로 해석이 가능하다.

In [22]:
selected = female[banker]
prob(selected)

0.7706043956043956

* 약 77% 의 은행원들이 여성이다.
* 이 조건부확률을 구하는 방식을 function으로 wrapping 하면 아래와 같다.
    * `conditional` 함수는 두 개의 Boolean series 인 `proposition`, `given`을 받는다. `given` 이 주어졌을 때 `proposition` 의 조건부확률을 구한다:

In [23]:
def conditional(proposition, given):
    """Probability of A conditioned on given."""
    return prob(proposition[given])

* 응답자가 여성일 때, 해당 응답자가 진보 성향일 확률은 아래와 같다.

In [25]:
conditional(liberal, given=female)

0.27581004111500884

* 함수의 given parameter 를 명시해줌으로써 가독성을 높일 수 있다.

## Conditional Probability Is Not Commutative
* 앞서 본 논리곱은 교환 가능한 성질을 가지고 있음을 확인했다. `prob(A & B)` 은 `prob(B & A)` 와 같다.
* 하지만 조건부확률은 교환 법칙이 성립하지 않는다; 즉 `conditional(A, B)` 은 `conditional(B, A)` 와 같지 않다.
* 예시를 통해 보면 쉽게 알 수 있다.

In [26]:
conditional(female, given=banker)

0.7706043956043956

* 위 결과는 응답자가 은행원일 때, 여성일 확률을 뜻한다. 반대로 응답자가 여성일 때 은행원일 확률은 아래와 같다.

In [27]:
conditional(banker, given=female)

0.02116102749801969

* 그 확률은 2% 밖에 되지 않다.
* 예시 자체는 확실하지만 `conditional(A, B)` 와 `conditional(B, A)` 는 혼동을 주기도 한다. 뒤의 예시에서 더 보기로 한다.

## Condition and Conjunction

* 논리곱과 조건부확률을 조합할 수 있다. 예를 들어, 응답자가 진보성향의 민주당원일 때 여성일 확률을 구해보자

In [28]:
conditional(female, given=liberal & democrat)

0.576085409252669

* 약 57% 의 진보성향의 민주당원은 여성이다.
* 은행원 중에서 진보성향의 민주당원을 구하면 아래와 같다.

In [29]:
conditional(liberal & female, given=banker)

0.17307692307692307

* 약 17% 의 은행원이 진보성향의 여성이다.

## Laws of Probability

이 절에서는 논리곱과 조건부확률 간의 세 가지 관계를 도출한다:

* Theorem 1: 논리곱을 사용한 조건부확률 계산.

* Theorem 2: 조건부확률을 사용한 논리곱 계산.

* Theorem 3: `conditional(A, B)` 를 사용한 `conditional(B, A)` 의 계산

* Theorem 3 은 베이즈이론 Bayes's Theorem 이라 불린다.

* using mathematical notation for probability:

    * $P(A)$ 는 사건 $A$ 에 대한 확률이다.

    * $P(A~\mathrm{and}~B)$ 는 사건 $A$와 사건 $B$의 논리곱의 확률이다. 즉, $A$와 $B$ 가 모두 참일 확률이다.

    * $P(A | B)$ 는 $B$ 가 주어졌을 때, $A$가 참이 ㄹ확률이다. $A$와 $B$ 사이의 세로선은 given 으로 읽는다.

### Theorem 1
* 은행원 중 몇 %가 여성인가:

1. 중괄호 [] 를 사용해 은행원을 선택하고

2. `mean` 함수로 은행원 중 여성의 비율을 구한다

In [30]:
female[banker].mean()

0.7706043956043956

* 혹은 `conditional` 함수를 쓴다.

In [31]:
conditional(female, given=banker)

0.7706043956043956

* 반면 두 확률에 대한 비율을 구해 조건부확률을 구하는 방법도 있다:

1. 응답자 중 여성 은행원의 비율을 구하고

2. 응답자 중 은행원의 비율을 구한다.

* 다시말해 , 전체 은행원 중 여성 은행원의 비율은 얼마나 되는지 구하는 것이다.

In [32]:
prob(female & banker) / prob(banker)

0.7706043956043956

* 결과는 위와 동일하다. 조건부확률과 논리곱을 결합하여 일반적인 규칙을 도출하기 위해 위 예시를 보았다. 수식은 아래와 같다:

$$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$$

Theorem 1 확인

### Theorem 2

* Theorem 1 에서 양변에 $P(B)$ 을 곱하면 Theorem 2이 도출된다.

$$P(A~\mathrm{and}~B) = P(B) ~ P(A|B)$$

* 이 식은 논리곱을 구하는 또 다른 방식을 보여준다. & 연산자 대신 두 확률을 곱하면 된다.
* `liberal` 과 `democrat` 에 적용해볼 때 `&`을 사용하면 아래와 같다.

In [33]:
prob(liberal & democrat)

0.1425238385067965

* Theorem 2 로 계산하면 아래와 같다.

In [34]:
prob(democrat) * conditional(liberal, democrat)

0.1425238385067965

### Theorem 3

아래 수식과 같이 논리곱은 교환 가능하다:

$$P(A~\mathrm{and}~B) = P(B~\mathrm{and}~A)$$

여기에 Theorem 2 적용하면 아래와 같이 도출된다

$$P(B) P(A|B) = P(A) P(B|A)$$

이는 다음과 같이 해석이 가능하다: 만약 $A$ and $B$ 을 확인하고 싶다면, 다음 중 하나를 선택하면 된다:

1. $B$ 먼저 확인하고, 그리고 $A$ conditioned on $B$ 를 살펴본다.

2. $A$ 먼저 확인하고, 그리고 $B$ conditioned on $A$ 를 살펴본다.

이를 $P(B)$ 로 나누면, Theorem 3 가 도출된다:

$$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$$

이것을 베이즈의 정리 (Bayes's Theorem)라 부른다. 

진보 성향의 은행원의 비율을 구하면서 이 수식이 어떻게 작동하는지 살표보자. `conditional`을 활용한다:

In [35]:
conditional(liberal, given=banker)

0.2239010989010989

using Bayes's Theorem:

In [36]:
prob(liberal) * conditional(banker, liberal) / prob(banker)

0.2239010989010989

위 두 결과가 동일함을 확인

### The Law of Total Probability

앞의 세 정리 외에도, 베이지안 통계를 사용하려면 한 가지의 정리가 더 필요하다. 이를 전확률의 법칙이라 하며 수식은 아래와 같다:

$$P(A) = P(B_1 \mathrm{and} A) + P(B_2 \mathrm{and} A)$$

즉, $A$ 의 전체 확률은 두 확률의 합이다: $B_1$ and $A$ 혹은 $B_2$ and $A$ 두 논리곱이 참인 경우이다.
하지만 이 법칙은 $B_1$ 와 $B_2$ 가 다음을 만족해야 ㅎ나다.:

* 상호 배타적 , 이는 전체 B 중 하나만 참인 경우이다.
* 전체 포괄, 이는 전체 B 중 하나는 반드시 참이다.

응답자가 은행원일 확률을 적용해 보자:

In [37]:
prob(banker)

0.014769730168391155

* 그럼 남성 은행원과 여성 은행원을 따로 계산했을 때 동일한 값이 나오는지 확인해보자.
* 여성 은행원에 대한 Series 는 이미 있기 때문에, 남성 은행원에 대한 Series 를 만들자

In [38]:
male = (gss['sex'] == 1)

두 확률을 더하면 `banker` 일 확률이 도출된다.

In [39]:
prob(male & banker) + prob(female & banker)

0.014769730168391155

* `male` 과 `female` 는 exclusive and collectively exhaustive (MECE) 속성이다. 
* Theorem 2 를 적용해, 전확률의 법칙을 아래와 같이 쓸 수 있다:

$$P(A) = P(B_1) P(A|B_1) + P(B_2) P(A|B_2)$$
* 또 이를 예시를 통해 다시 확인이 가능하다.

In [40]:
(prob(male) * conditional(banker, given=male) +
prob(female) * conditional(banker, given=female))

0.014769730168391153

두 개 이상의 condidtion (조건부확률의 조건), 이 있는 경우에는 아래와 같이 summation 으로 전확률의 법칙을 표현할 수 있다:

$$P(A) = \sum_i P(B_i) P(A|B_i)$$

단, 위에서 언급한바와 같이 조건에 해당하는 사건들은 MECE 조건을 만족해야 한다.
예를 들어 정치적 성향은 1~7 로 구분한 것 처럼 말이다.

In [41]:
B = gss['polviews']
B.value_counts().sort_index()

1.0     1442
2.0     5808
3.0     6243
4.0    18943
5.0     7940
6.0     7319
7.0     1595
Name: polviews, dtype: int64

중도 성향의 은행원이 응답할 확률은 아래와 같다.

In [42]:
i = 4
prob(B==i) * conditional(banker, B==i)

0.005822682085615744

파이썬의 `sum` 함수와 [generator expression](https://www.johndcook.com/blog/2020/01/15/generator-expression/) 을 활용하면 전확률의 법칙을 계산할 수 있다.

In [44]:
sum(prob(B==i) * conditional(banker, B==i)
    for i in range(1, 8)) # 응답자가 은행원일 확률

0.014769730168391157

## Summary

이번 Chapter 의 내용을 요약해보면 다음과 같다.

**Theorem 1** 논리곱을 사용해 조건부확률을 구하는 방법:

$$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$$

**Theorem 2** 조건부확률을 사용해 논리곱을 구하는 방법:

$$P(A~\mathrm{and}~B) = P(B) P(A|B)$$

**Theorem 3**, 베이즈 정리라고 하며, $P(A|B)$ 나 $P(A|B)$ 가 주어졌을 때 원래의 확률을 찾는 방법이다:

$$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$$

**The Law of Total Probability** 각각의 확률을 더해서 전체확률을 구하는 방법:

$$P(A) = \sum_i P(B_i) P(A|B_i)$$

## Exercises

**Exercise:** Let's use the tools in this chapter to solve a variation of the Linda problem.

> Linda is 31 years old, single, outspoken, and very bright. She majored in philosophy. As a student, she was deeply concerned with issues of discrimination and social justice, and also participated in anti-nuclear demonstrations.  Which is more probable?
> 1. Linda is a banker.
> 2. Linda is a banker and considers herself a liberal Democrat.

To answer this question, compute 

* The probability that Linda is a female banker,

* The probability that Linda is a liberal female banker, and

* The probability that Linda is a liberal female banker and a Democrat.

In [47]:
# Solution goes here
prob(female & banker)

0.011381618989653074

In [48]:
# Solution goes here
prob(female & banker & liberal)

0.002556299452221546

In [49]:
# Solution goes here
prob(female & banker & liberal & democrat)

0.0012375735443294787

**Exercise:** Use `conditional` to compute the following probabilities:

* What is the probability that a respondent is liberal, given that they are a Democrat?

* What is the probability that a respondent is a Democrat, given that they are liberal?

Think carefully about the order of the arguments you pass to `conditional`.

In [51]:
# Solution goes here
conditional(liberal,given=democrat)

0.3891320002215698

In [52]:
# Solution goes here
conditional(democrat,given=liberal)

0.5206403320240125

**Exercise:** There's a [famous quote](https://quoteinvestigator.com/2014/02/24/heart-head/) about young people, old people, liberals, and conservatives that goes something like:

> If you are not a liberal at 25, you have no heart. If you are not a conservative at 35, you have no brain.

Whether you agree with this proposition or not, it suggests some probabilities we can compute as an exercise.
Rather than use the specific ages 25 and 35, let's define `young` and `old` as under 30 or over 65:

In [53]:
young = (gss['age'] < 30)
prob(young)

0.19435991073240008

In [54]:
old = (gss['age'] >= 65)
prob(old)

0.17328058429701765

For these thresholds, I chose round numbers near the 20th and 80th percentiles.  Depending on your age, you may or may not agree with these definitions of "young" and "old".

I'll define `conservative` as someone whose political views are "Conservative", "Slightly Conservative", or "Extremely Conservative".

In [55]:
conservative = (gss['polviews'] >= 5)
prob(conservative)

0.3419354838709677

Use `prob` and `conditional` to compute the following probabilities.

* What is the probability that a randomly chosen respondent is a young liberal?

* What is the probability that a young person is liberal?

* What fraction of respondents are old conservatives?

* What fraction of conservatives are old?

For each statement, think about whether it is expressing a conjunction, a conditional probability, or both.

For the conditional probabilities, be careful about the order of the arguments.
If your answer to the last question is greater than 30%, you have it backwards!

In [56]:
# Solution goes here
prob(young&liberal)

0.06579427875836884

In [57]:
# Solution goes here
conditional(liberal, given=young)

0.338517745302714

In [58]:
# Solution goes here
prob(old&conservative)

0.06701156421180766

In [59]:
# Solution goes here
conditional(old, given=conservative)

0.19597721609113564